<a href="https://colab.research.google.com/github/PHaasbroek/Google-Colab/blob/main/Fluid-Solver/Fluid-Solver-Cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
# Header files need to be written at every compile to implement changes made to the header.

%%writefile functions.h

void printFunction();
double Re(double density, double dynamicViscosity, double charLength, double charSpeed);
double skinFriction_Schultz_Grunov(double Re);
double wallShearStress_f(double skinFriction, double density, double meanVelocity);
double frictionVelocity_f(double wallShearStress, double density);
double dimensionlessFirstCellHeight_f(double density, double frictionVelocity, double firstCellHeight, double dynamicViscosity);


void printFunction(){
  std::cout << "Function Works 3" << std::endl;
}

double Re_f(double density, double dynamicViscosity, double charLength, double charSpeed){
  // density [kg/m^3]
  // dynamicViscosity [Pa * s] = [kg/m/s]
  // charLength [m]
  // charSpeed [m.s]

  return density * charSpeed * charLength / dynamicViscosity;
}


double skinFriction_Schultz_Grunov_f(double Re){
  // https://www.cfd-online.com/Wiki/Skin_friction_coefficient#References
  // Schultz-Grunov (equation 21.19a in Schlichting, Hermann (1979), Boundary Layer Theory, ISBN 0-07-055334-3, 7th Edition.)
  // skin friction on a flat plate for turbulent flow.
  // skinFriction [-]

  return 0.370 * pow(log10(Re),-2.584);
}

double wallShearStress_f(double skinFriction, double density, double meanVelocity){
  // wallShearStress [N/m^2]
  return 0.5 * skinFriction * density * meanVelocity * meanVelocity;
}

double frictionVelocity_f(double wallShearStress, double density){
  // wallShearStress [N/m^2]
  // density [kg/m^3]

  return pow(wallShearStress / density, 0.5);
}

double dimensionlessFirstCellHeight_f(double density, double frictionVelocity, double firstCellHeight, double dynamicViscosity){
  // density [kg/m^3]
  // frictionVelocity [m/s]
  // firstCellHeight [m]
  // dynamicViscosity [Pa * s] = [kg/m/s]

  return density * frictionVelocity * firstCellHeight / dynamicViscosity;
}

double sumOfGeometricSeries_f(double firstEntry = 1.0, double ratio = 1.0, int numExpandingCells = 1){
  // assume that consecutive cells have a ratio > 1.0
  // assume that the cells at the walls are the smallest

  if(ratio == 1.0){
    ratio = ratio + 1e-9;
  }

  return firstEntry * (1.0 - pow(ratio, numExpandingCells)) / (1.0 - ratio);
}





Overwriting functions.h


In [105]:
#Testing the Fluid-Solver-Cpp

%%writefile FluidSolverCpp.cpp

// Standard Libraries: ////////////////////////
#include <iostream>
#include <cmath>

// Custom headers files: /////////////////////////////
#include "functions.h"



int main(void) {

  // Input Parameters: ///////////////////////
  int cellNumberHeight = 100; // number of cells through the height of the domain
  double cellWidth = 1.0; // unit width of cells
  double firstCellHeight = 8e-5; // [m]
  double diameter = 0.050; // [m]
  double meanVelocity = 2.0; // [m/s]
  double density = 1000.0; // [kg/m^3]
  double dynamicViscosity = 0.01; // [Pa * s]

  double Re = Re_f(density, dynamicViscosity, diameter, meanVelocity);
  double skinFriction = skinFriction_Schultz_Grunov_f(Re);
  double wallShearStress = wallShearStress_f(skinFriction, density, meanVelocity);
  double frictionVelocity = frictionVelocity_f(wallShearStress, density);
  double dimensionlessFirstCellHeight = dimensionlessFirstCellHeight_f(density, frictionVelocity, firstCellHeight, dynamicViscosity);


  std::cout << "Reynolds number:" << Re << " [-]\n";
  std::cout << "Skin friction: " << skinFriction << " [-]\n";
  std::cout << "Wall shear stress: " << wallShearStress << " [Pa]\n";
  std::cout << "Friction velocity: " << frictionVelocity << " [m/s]\n";
  std::cout << "Dimensionless first cell height: " << dimensionlessFirstCellHeight << " [-]\n";
  std::cout << std::endl;




  return 0;
  }


Overwriting FluidSolverCpp.cpp


In [106]:
%%script bash
g++ FluidSolverCpp.cpp -o FluidSolverCpp
./FluidSolverCpp

Reynolds number:10000 [-]
Skin friction: 0.0102915 [-]
Wall shear stress: 20.583 [Pa]
Friction velocity: 0.143468 [m/s]
Dimensionless first cell height: 1.14774 [-]

7